# Librerie utilizzate

In [148]:
from pulp import * # per modellare il problema e risolverlo
import numpy as np # per creare array
import math #per arrotondamenti superiori

# Master problem o problema ristretto

In [149]:
class MasterProblem:
	def __init__(self, lunghezza_max, lunghezza_assi, domanda_assi, patterns):
		
		self.lunghezza_max=lunghezza_max
		self.lunghezze_assi=lunghezza_assi
		self.domande_assi=domanda_assi
		self.patterns= patterns
		
		self.prob = LpProblem('MasterProblem',LpMinimize)
		self.obj = LpConstraintVar("obj")
		self.prob.setObjective(self.obj)
		
		# generazione lista di vincoli
		self.PatternVars=[]
		self.lista_vincoli=[]
		for index,value in enumerate(domanda_assi):
			var=LpConstraintVar("C"+str(index),LpConstraintGE,value)  # generazione vincolo a_ij * x_j >= d_i
			self.lista_vincoli.append(var)
			self.prob+=var

		# generazione della matrice A quindi i valori a_ij
		for i,x in enumerate(self.patterns):
			temp=[]
			for j,y in enumerate(x):
				# solo quelle >0 per creare la variabile
				if y>0: 
					temp.append(j)

			# generazione di variabili con solo coeffiecienti > 0, cioè quanti pezzi di un certo tipo per il pattern i-esimo
			var=LpVariable("x"+str(i + 1), 0, None, LpContinuous, lpSum(self.obj+[self.lista_vincoli[v] for v in temp]))
			self.PatternVars.append(var)
		
	# risolvi il problem master restituendo la soluzione duale u*
	def risolvi(self):
		self.prob.writeLP('master.lp')
		self.prob.solve()
		return [self.prob.constraints[i].pi for i in self.prob.constraints]
		
			
	# column generation => aggiungi il pattern al problema ristretto
	def aggiungiPattern(self,pattern):
		
		self.patterns.append(pattern)
		temp=[]
		
		for j,y in enumerate(pattern):
			if y>0: 
				temp.append(j)

		# aggiungi le variabili con lo stesso criterio di prima
		var=LpVariable("x"+str(len(self.patterns) + 1)	, 0, None, LpContinuous, lpSum(self.obj+[pattern[v]*self.lista_vincoli[v] for v in temp]))
		self.PatternVars.append(var)

	def getOttimo(self):
		return value(self.prob.objective)

# Slave problem

In [150]:
class SlaveProblem:
    def __init__(self,duale, itemLengths,maxValue):
        self.prob=LpProblem("SlaveProblem",LpMaximize)
        self.lista_var=[LpVariable('q'+str(i + 1),0,None,LpInteger) for i in range(len(duale))]
        self.prob+=lpSum([duale[i]*x for i,x in enumerate(self.lista_var)])  #use duals to set objective coefficients
        self.prob+=lpSum([itemLengths[i]*x for i,x in enumerate(self.lista_var)])<=maxValue 

        self.prob.writeLP('slave.lp')
        self.prob.solve() 

    def risolviSlave(self):
        pattern=False
        # TEST OTTIMALITA'
        if value(self.prob.objective) > 1:
            pattern=[]
            for v in self.lista_var:
                pattern.append(value(v))
        return pattern

# Risoluzione di un'istanza

In [151]:
nr_assi=5
lunghezza_max_asse=110


lunghezza_assi=[20, 45, 50, 55, 75]
domanda_assi=[48, 35, 24, 10, 8]


if len(lunghezza_assi) != len(domanda_assi):
	print('ERRORE', "la cardinalità dell'insieme lunghezza e domanda non è uguale")
else:
	nr_assi = len(lunghezza_assi)
	
print("Lunghezze assi : %s" % lunghezza_assi)
print("Domande assi : %s\n" % domanda_assi)


# soluzione di partenza: usare patterns con un solo asse per ogni i-esimo tipo
patterns=[]
for i in range(nr_assi):
	pattern=np.zeros(nr_assi)
	pattern[i] = 1.0
	patterns.append(pattern)

masterProblem=MasterProblem(lunghezza_max_asse,lunghezza_assi,domanda_assi,patterns)
	
solOttima=False
while solOttima==False:   # once no more new columns can be generated set relaxed to False

	duale=masterProblem.risolvi()
	slaveProblem=SlaveProblem(duale,lunghezza_assi,lunghezza_max_asse)
	newPattern=slaveProblem.risolviSlave()
	
	if newPattern:
		masterProblem.aggiungiPattern(newPattern)
	else:
		solOttima = True
		masterProblem.risolvi()
		solOttima=True

ottimo = math.ceil(masterProblem.getOttimo())
print("Ottimo: %s assi da tagliare" % ottimo)


Lunghezze assi : [20, 45, 50, 55, 75]
Domande assi : [48, 35, 24, 10, 8]

Ottimo: 47 assi da tagliare
